In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from utility import calculate_input_gradients, perturb_inputs, preprocess_images, \
                    postprocess_features, save_data_hdf5,get_dataset_hdf5,\
                    build_one_class_svm, combine_inliners_outliers, apply_temp_scale_to_model,\
                    apply_log_temp_scale_to_model, perturb_inputs_odin, extract_layer_features
 
from utility_db_outliers import load_dataset
from models_lib import load_custom_model_for_ds
import h5py
# from metrics2 import *
from metrics import *
from general_setting import *
import time
import tensorflow as tf
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras.layers import Activation
from tensorflow.keras import layers
from tensorflow.keras.models import load_model
#------------
from utility_methods import *
# from utility_methods2 import *
from sklearn.metrics import roc_auc_score

from tensorflow.keras.utils import to_categorical
import gzip
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.datasets import mnist
import scipy.io as sio
import tensorflow.keras.backend as K
import math

/home/zhaofz/anaconda3/envs/tf-gpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/zhaofz/anaconda3/envs/tf-gpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/zhaofz/anaconda3/envs/tf-gpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/zhaofz/anaconda3/envs/tf-gpu/lib/python3

In [2]:
import matplotlib.pyplot as plt
from scipy.stats import norm
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.layers import Conv2D, Flatten, Lambda
from tensorflow.keras.layers import Reshape, Conv2DTranspose, MaxPooling2D, UpSampling2D, AveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K

In [3]:
BATCH_SIZE = 100
SAVE_RESULTS = True
id_name=ID_DS_LIST[2] # selects the ID dataset.
id_model=ID_MODEL_LIST[2]  # select the deep model used for training ID dataset.
print(id_name,id_model)

SVHN ResNet


In [4]:
(org_traing_data, org_training_labels),(id_eva_data,org_testing_labels)  = load_dataset(id_name)

org_traing_data_processed = preprocess_images(id_name, org_traing_data, id_model, verbose=True)
id_eva_data_processed = preprocess_images(id_name, id_eva_data, id_model, verbose=True)

Preprocessing was done for  SVHN ResNet
Preprocessing was done for  SVHN ResNet


In [5]:
x_train = org_traing_data_processed
y_train = org_training_labels
x_test = id_eva_data_processed
y_test = org_testing_labels

In [6]:
# 网络参数
input_shape = (32, 32, 3)
batch_size = 256
kernel_size = 3
# filters = 16  
latent_dim = 1024 # 隐变量取2维只是为了方便后面画图
epochs = 100
x_in = Input(shape=input_shape)
x = x_in

In [7]:
structure = [32, 64, 128]

In [8]:
for filters in structure:  
    if isinstance(filters, int):
        x = Conv2D(filters=filters,
                kernel_size=kernel_size,
                activation='relu',
                strides=2,
                padding='same')(x)
    # elif filters == "max":
    #     x = MaxPooling2D((2, 2), padding="same")(x)
    # elif filters == "average":
    #     x = AveragePooling2D((2, 2), padding="same")(x)

# 备份当前shape，等下构建decoder的时候要用
shape = K.int_shape(x)

x = Flatten()(x)
x = Dense(4*4*128, activation='relu')(x)
# 算p(Z|X)的均值和方差
z_mean = Dense(latent_dim)(x)
z_log_var = Dense(latent_dim)(x)

# 重参数技巧
def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=K.shape(z_mean))
    return z_mean + K.exp(z_log_var / 2) * epsilon

In [9]:
# 重参数层，相当于给输入加入噪声
z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

# 解码层，也就是生成器部分
# 先搭建为一个独立的模型，然后再调用模型
latent_inputs = Input(shape=(latent_dim,))
x = Dense(shape[1] * shape[2] * shape[3], activation='relu')(latent_inputs)
x = Reshape((shape[1], shape[2], shape[3]))(x)

In [10]:
for filters in structure:  
    if isinstance(filters, int):
        x = Conv2DTranspose(filters=filters,
                    kernel_size=kernel_size,
                    activation='relu',
                    strides=2,
                    padding='same')(x)
    # elif filters == "max" or filters == "average":
    #             x = UpSampling2D((2, 2))(x)

outputs = Conv2DTranspose(filters=3,
                        kernel_size=kernel_size,
                        activation='sigmoid',
                        padding='same')(x)

In [11]:
# 搭建为一个独立的模型
encoder = Model(x_in, z_mean)

decoder = Model(latent_inputs, outputs)

x_out = decoder(z)

In [12]:
# 建立模型
vae = Model(x_in, x_out)

In [13]:
# xent_loss是重构loss，kl_loss是KL loss
xent_loss = K.sum(K.binary_crossentropy(x_in, x_out), axis=[1, 2, 3])
kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
vae_loss = K.mean(xent_loss + kl_loss)

# add_loss是新增的方法，用于更灵活地添加各种loss
vae.add_loss(vae_loss)
vae.compile(optimizer='rmsprop')
vae.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 16, 16, 32)   896         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 8, 8, 64)     18496       conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 4, 4, 128)    73856       conv2d_1[0][0]                   
__________________________________________________________________________________________________
flatten (F

In [14]:
vae.fit(x_train,
        shuffle=True,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(x_test, None))

Train on 73257 samples, validate on 26032 samples
Epoch 1/100
73257/73257 [==============================] - 13s 178us/step - loss: 2040.1249 - val_loss: 1926.9146
Epoch 2/100
73257/73257 [==============================] - 11s 144us/step - loss: 1987.2287 - val_loss: 1928.0178
Epoch 3/100
73257/73257 [==============================] - 10s 142us/step - loss: 1975.8592 - val_loss: 1930.3512
Epoch 4/100
73257/73257 [==============================] - 10s 142us/step - loss: 1967.7676 - val_loss: 1898.4895
Epoch 5/100
73257/73257 [==============================] - 10s 141us/step - loss: 1963.4664 - val_loss: 1917.2232
Epoch 6/100
73257/73257 [==============================] - 10s 142us/step - loss: 1959.8447 - val_loss: 1902.4656
Epoch 7/100
73257/73257 [==============================] - 11s 144us/step - loss: 1956.9241 - val_loss: 1891.2253
Epoch 8/100
73257/73257 [==============================] - 10s 143us/step - loss: 1954.3282 - val_loss: 1906.8575
Epoch 9/100
73257/73257 [=============

In [15]:
vae.save('./vae_models/%s_%s.h5'%(id_name,id_model))
vae.save_weights('./vae_models/%s_%s_weights.h5'%(id_name,id_model))

In [16]:
def mark(x_1, x_2):
    diff = np.abs(x_1 - x_2)
    marks = np.mean(np.power(diff, 2), axis=(1,2,3))
    return marks

In [22]:
x_test = np.load('./advs_new/%s_%s_normal.npy'%(id_name, id_model),allow_pickle=True)
print(x_test.shape)

(25022, 32, 32, 3)


In [23]:
sc_id = []
for i in range(len(x_test)):
    x_test_encoded = encoder.predict(x_test[i].reshape(1,32,32,3))
    xx = decoder.predict(x_test_encoded)
    score = mark(x_test[i],xx)
    # print(score)
    sc_id.append(-score)

In [18]:
def combine_inliners_outliers(inliers, outliers, i_label=1, o_label=0, verbose=False):

    temp_outliers = outliers
    temp_inliers  = inliers
    if i_label==1:
        i_labels = np.ones(temp_inliers.shape[0])
    else:
        i_labels = np.zeros(temp_inliers.shape[0])
        
    if o_label==0:
        o_labels = np.zeros(temp_outliers.shape[0])
    else:
        o_labels = np.ones(temp_outliers.shape[0])       
              
    mixed_labels =  np.append(i_labels, o_labels)
    mixed_data = np.vstack((temp_inliers, temp_outliers))

    return mixed_data, mixed_labels

In [24]:
test = np.asarray(sc_id)
ADV_DS_LIST = ("fgsm", "bim-a", "bim-b", "jsma", "cw-l2")
for i, ds_name in enumerate(ADV_DS_LIST):
    adv = id_name+"_"+id_model+"_"+ ds_name
    das = np.load('./advs_new/'+adv+'.npy',allow_pickle=True)
    sc_ood = []
    for i in range(len(das)):
        x_test_encoded = encoder.predict(das[i].reshape(1,32,32,3))
        xx = decoder.predict(x_test_encoded)
        score = mark(das[i],xx)
        sc_ood.append(-score)
    
    ood = np.asarray(sc_ood)
    scores, mixed_labels = combine_inliners_outliers(test, ood)
 
    fpr, tpr = nums(scores, mixed_labels)
    roc_auc = auc(fpr, tpr)
    lens = id_eva_data_processed.shape[0]
    FP,TN,TP,FN = ErrorRateAt95Recall1(lens, scores, mixed_labels)
    ROC = roc_auc_score(mixed_labels, scores, average='micro', sample_weight=None)
    #     print(get_summary_statistics(scores, mixed_labels))
    print('The results of %s:\n' %(ds_name))
    #     print('fpr_at_95_tpr:', float(FP) / float(FP + TN+ 1e-7) *100)
    print('tnr_at_95_tpr:', float(TN) / float(FP + TN+ 1e-7) *100)
    print('detection_acc:',(float(TP) / float(TP + FN + 1e-7)+ float(TN) / float(FP + TN + 1e-7))/2*100)
    #     print('detection_errror:',(1.0- float(TP) / float(TP + FN+ 1e-7)+ float(FP) / float(FP + TN+ 1e-7))/2*100)
    print("AUROC: ",ROC*100)
    print("\n")

The results of fgsm:

tnr_at_95_tpr: 91.2502082288387
detection_acc: 93.12433582904941
AUROC:  99.01038856248512


The results of bim-a:

tnr_at_95_tpr: 5.289022155566846
detection_acc: 50.14374279241349
AUROC:  52.80341693522792


The results of bim-b:

tnr_at_95_tpr: 15.900383141696233
detection_acc: 55.4494232854782
AUROC:  86.33904042789399


The results of jsma:

tnr_at_95_tpr: 16.73330001658865
detection_acc: 55.86588172292441
AUROC:  77.22924983396116


The results of cw-l2:

tnr_at_95_tpr: 4.955855405609879
detection_acc: 49.97715941743501
AUROC:  50.16825965271096




In [19]:
#layer_inx specifies the index of the OODL found by "find_oodl" jupyter file.
REM_TOP_LAYER = -2
NUM_CLASS = 10
if id_name=="MNIST" and id_model=="LeNet":
    layer_inx = 0
    OOD_DS_LIST       = OOD_DS_LIST_MNIST
    #**********************************************************
elif id_name=="CIFAR10":
    OOD_DS_LIST      = OOD_DS_LIST_CIFAR10
    if id_model=="VGG":
        layer_inx = 6
    elif id_model=="ResNet":
        layer_inx = 20
    #**********************************************************
elif id_name=="SVHN":
    OOD_DS_LIST      = OOD_DS_LIST_SVHN
    if id_model=="VGG":
        layer_inx = 6
    elif id_model=="ResNet":
        layer_inx = 24

In [20]:
# Vriance模块(OODL大类别 --> 删除同样的feature maps)
test = np.asarray(sc_id)
for i, ood_ds_name in enumerate(OOD_DS_LIST):
    (_,_),(ood_eva_data,_) = load_dataset(ood_ds_name)
    das = preprocess_images(id_name, ood_eva_data, id_model, verbose=True)
    
    # scaler.fit(das.flatten().reshape(das.shape[0],das.shape[1]*das.shape[2]*das.shape[3]))
    # das_0 = scaler.transform(das.flatten().reshape(das.shape[0],das.shape[1]*das.shape[2]*das.shape[3]))
    # das = das_0.reshape(das.shape[0],das.shape[1],das.shape[2],das.shape[3])
    
    sc_ood = []
    for i in range(len(das)):
        x_test_encoded = encoder.predict(das[i].reshape(1,32,32,3))
        xx = decoder.predict(x_test_encoded)
        score = mark(das[i],xx)
        sc_ood.append(-score)
    
    ood = np.asarray(sc_ood) 
    scores, mixed_labels = combine_inliners_outliers(test, ood) 

    fpr, tpr = nums(scores, mixed_labels)
    roc_auc = auc(fpr, tpr)
    lens = id_eva_data_processed.shape[0]
    FP,TN,TP,FN = ErrorRateAt95Recall1(lens, scores, mixed_labels)
    ROC = roc_auc_score(mixed_labels, scores, average='micro', sample_weight=None)
#     print(get_summary_statistics(scores, mixed_labels))
    print('The results of %s:\n' %(ood_ds_name))
#     print('fpr_at_95_tpr:', float(FP) / float(FP + TN+ 1e-7) *100)
    print('tnr_at_95_tpr:', float(TN) / float(FP + TN+ 1e-7) *100)
    print('detection_acc:',(float(TP) / float(TP + FN + 1e-7)+ float(TN) / float(FP + TN + 1e-7))/2*100)
#     print('detection_errror:',(1.0- float(TP) / float(TP + FN+ 1e-7)+ float(FP) / float(FP + TN+ 1e-7))/2*100)
    print("AUROC: ",ROC*100)
    print("\n")

Preprocessing was done for  SVHN ResNet
The results of TINYIMAGENET_RESIZED_32:

tnr_at_95_tpr: 94.86999999905129
detection_acc: 94.93423171415571
AUROC:  98.82915507836509


Preprocessing was done for  SVHN ResNet
The results of LSUN_RESIZED:

tnr_at_95_tpr: 98.499999999015
detection_acc: 96.74923171413758
AUROC:  99.51437346342964


Preprocessing was done for  SVHN ResNet
The results of ISUN_PATCHED:

tnr_at_95_tpr: 96.16806722581323
detection_acc: 95.5832653275367
AUROC:  99.10810009314113


Preprocessing was done for  SVHN ResNet
The results of CIFAR10:

tnr_at_95_tpr: 80.17999999919819
detection_acc: 87.58923171422917
AUROC:  96.19687768899816


Preprocessing was done for  SVHN ResNet
The results of G_255:

tnr_at_95_tpr: 99.99999999899998
detection_acc: 97.49923171413006
AUROC:  100.0


Preprocessing was done for  SVHN ResNet
The results of U_255:

tnr_at_95_tpr: 99.99999999899998
detection_acc: 97.49923171413006
AUROC:  100.0




In [21]:
ood_ds_name = 'foolingimages'

das = np.load('adv_datasets/%s_%s_fooling_images.npy'%(id_name,id_model))

# scaler.fit(das.flatten().reshape(das.shape[0],das.shape[1]*das.shape[2]*das.shape[3]))
# das_0 = scaler.transform(das.flatten().reshape(das.shape[0],das.shape[1]*das.shape[2]*das.shape[3]))
# das = das_0.reshape(das.shape[0],das.shape[1],das.shape[2],das.shape[3])

sc_ood = []
for i in range(len(das)):
        x_test_encoded = encoder.predict(das[i].reshape(1,32,32,3))
        xx = decoder.predict(x_test_encoded)
        score = mark(das[i],xx)
        sc_ood.append(-score)

ood = np.asarray(sc_ood) 
scores, mixed_labels = combine_inliners_outliers(test, ood) 

# print('scores is:', (scores[10000:20000]))
fpr, tpr = nums(scores, mixed_labels)
roc_auc = auc(fpr, tpr)
lens = id_eva_data_processed.shape[0]
FP,TN,TP,FN = ErrorRateAt95Recall1(lens,scores, mixed_labels)
ROC = roc_auc_score(mixed_labels, scores, average='micro', sample_weight=None)
print('The results of %s:\n' %(ood_ds_name))
# print(get_summary_statistics(scores, mixed_labels))
# print('fpr_at_95_tpr:', float(FP) / float(FP + TN+ 1e-7) *100)
print('tnr_at_95_tpr:', float(TN) / float(FP + TN+ 1e-7) *100)
print('detection_acc:',(float(TP) / float(TP + FN + 1e-7)+ float(TN) / float(FP + TN + 1e-7))/2*100)
# print('detection_errror:',(1.0- float(TP) / float(TP + FN+ 1e-7)+ float(FP) / float(FP + TN+ 1e-7))/2*100)
print("AUROC: ",ROC*100)
print("\n")

The results of foolingimages:

tnr_at_95_tpr: 99.99999999899998
detection_acc: 97.49923171413006
AUROC:  100.0


